In [54]:
import csv
import math
import time
import itertools
import warnings
import operator
import statistics
import seaborn as sns
import pandas as pd
import numpy as np
import geopy.distance
import matplotlib.pyplot as plt
from scipy.stats import linregress

import datetime as dt 
from dateutil.parser import parse
from datetime import datetime, timedelta
from scipy import stats
from IPython.display import display_html
from multiprocessing import Pool
from sklearn.model_selection import train_test_split

plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

In [87]:
df_ads = pd.read_csv('gs://aes-analytics-0002-curated/Outage_Restoration/DPL_Curated_Datasets/DPL_Master_dataset_storm_all_blue_3yr_v2.3_.csv')
df_ads.drop(columns= ['Unnamed: 0'], inplace=True)
df_ads['TIME_OF_FIRST_CALL'] = pd.to_datetime(df_ads['TIME_OF_FIRST_CALL'], infer_datetime_format = True, errors = 'coerce')
df_ads['TIME_TO_RESTORE'] = df_ads['TIME_OF_FIRST_CALL'] + pd.to_timedelta(df_ads['TTR']*60, unit='s')
df_ads.reset_index(drop=True, inplace=True)
df_ads.head()

,OUTAGE_ID,CIRCUIT,DEVICE,TIME_OF_FIRST_CALL,PHASE_A,PHASE_B,PHASE_C,PHASE_T,LFS_CUSTS,MED_CUSTS,EMR_CUSTS,UTL_CUSTS,MAJ_CUSTS,GOV_CUSTS,LRT_CUSTS,WIREDOWN_PTH,WIREDOWN_PTP,TREEONWIRE_PTH,TREEONWIRE_PTP,BROKENPOLE,FIREEXPLOSION,DAMAGEDTRANS,PRIORITY,CONFIDENCE,CUSTSTILLOUT,TTR,MAJOR,TORD_False,TORD_True,ACCOUNT_TYPE_COM,ACCOUNT_TYPE_CUS,ACCOUNT_TYPE_GOV,ACCOUNT_TYPE_IND,ACCOUNT_TYPE_RES,STORM_TYPE,EVENT,Day_flag,Hour_Sin,Hour_Cos,DATE,RANK_SUBSEQUENT_OTG,weekday,Weekend_flag,Marker_Location,Outages_in_last_1hr,Outages_in_last_2hr,Outages_in_last_3hr,Outages_in_last_4hr,Outages_in_last_5hr,Outages_in_last_6hr,Outages_in_last_7hr,Outages_in_last_8hr,Outages_in_last_9hr,Outages_in_last_10hr,Min_Distance_service_Center,Nearest_service_center,cldCvrMin,cldCvrAvg,cldCvrMax,dewPtMin,dewPtAvg,dewPtMax,feelsLikeMin,feelsLikeAvg,feelsLikeMax,heatIndexMin,heatIndexAvg,heatIndexMax,mslPresMin,mslPresAvg,mslPresMax,precip,radSolarMin,radSolarAvg,radSolarMax,radSolarTot,relHumMin,relHumAvg,relHumMax,sfcPresMin,sfcPresAvg,sfcPresMax,snowfall,spcHumMin,spcHumAvg,spcHumMax,tempMin,tempAvg,tempMax,windChillMin,windChillAvg,windChillMax,windDirAvg,windDir80mAvg,windDir100mAvg,windSpdMin,windSpdAvg,windSpdMax,windSpd80mMin,windSpd80mAvg,windSpd80mMax,windSpd100mMin,windSpd100mAvg,windSpd100mMax,wetBulbMin,wetBulbAvg,wetBulbMax,tempRange,windSpdRange,sfcPresRange,cldCvrRange,relHumRange,relHumRatio,sfcPresRatio,DEVICETYPE,TOPFUSE,TOTCUST_A,TOTCUST_B,TOTCUST_C,TOTCUST_T,VOLTAGE,UNDERGD,NORMALLYOPEN,TOTKVA_A,TOTKVA_B,TOTKVA_C,Total_Crew_dispatched,Total_Crew_available,Line_crew,Service_crew,Substation_crew,Network_crew,Contractor,rank,rank_cd,Slope_outages,Slope_ROC_outages,max_ROC_outages,max_index_ROC,weight_ROC,Total_Customers,Total_Key_Customers,Month_Sin,Month_Cos,Pole,Nearest_service_center_mod,TIME_TO_RESTORE
0,598691,RJ1205,09851,2007-02-13 01:34:17,True,False,False,False,0.00,0.00,0.00,0.00,0.00,0.00,0.00,False,False,False,False,False,False,True,False,H,0,351.40,0.00,8.00,0.00,False,False,False,False,False,Ice,STORM,False,0.26,0.97,2007-02-13,1.00,Tuesday,False,Marker 25,0,0,0,0,0,2,3,4,4,4,9.59,DSB,99.00,100.00,100.00,12.50,23.40,26.90,0.00,14.10,19.10,17.30,27.60,30.90,1006.50,1013.20,1021.30,1.31,0.00,29.70,154.90,711.90,77.40,84.00,90.50,972.70,979.10,986.80,10.80,1.70,2.80,3.10,17.30,27.60,30.90,0.00,14.10,19.10,52.00,54.00,54.00,14.30,20.30,23.30,16.90,22.70,25.70,16.60,22.50,25.50,16.10,26.20,29.20,13.60,9.00,14.10,1.00,13.10,1.17,1.01,FUSE,False,45.00,0.00,0.00,0.00,12KV (100%),False,False,387.00,0.00,0.00,0,13,11,2,0,0,1,1.00,1.00,1.48,0.71,2,5,1.00,45.00,0.00,0.50,0.87,False,Center,2007-02-13 07:25:41.000
1,598695,HD1202,01986,2007-02-13 07:02:48,False,False,True,False,0.00,0.00,0.00,0.00,0.00,0.00,0.00,False,False,False,False,False,False,False,False,H,0,188.82,0.00,3.00,0.00,False,False,False,False,False,Ice,STORM,True,0.97,-0.26,2007-02-13,2.00,Tuesday,False,Marker 41,0,0,0,0,0,1,1,1,1,1,44.02,Washington Court House,100.00,100.00,100.00,18.80,27.30,28.80,7.60,19.00,22.00,22.70,30.80,33.20,1004.40,1011.70,1020.80,1.39,0.00,29.40,140.00,705.10,80.90,86.90,91.00,967.40,974.70,983.00,0.50,2.30,3.20,3.40,22.70,30.80,33.20,7.60,19.00,22.00,54.00,56.00,56.00,13.80,18.20,21.10,16.00,20.20,23.10,15.80,20.00,23.00,21.50,29.50,31.30,10.50,7.30,15.60,0.00,10.10,1.12,1.02,FUSE,False,0.00,0.00,5.00,0.00,12KV (100%),False,False,0.00,0.00,75.00,2,22,20,2,0,0,1,1.00,1.00,5.00,-0.00,1,5,0.50,5.00,0.00,0.50,0.87,False,Not_Center,2007-02-13 10:11:37.200
2,598696,CF1203,D8994,2007-02-13 07:33:14,False,False,True,False,0.00,0.00,0.00,0.00,0.00,0.00,0.00,False,False,False,False,False,False,False,False,H,0,87.47,0.00,2.00,0.00,False,False,False,False,False,Ice,STORM,True,0.97,-0.26,2007-02-13,3.00,Tuesday,False,Marker 13,1,1,1,1,1,2,2,2,2,2,19.70,Marysville,100.00,100.00,100.00,11.80,18.00,23.90,-1.60,6.70,16.40,16.20,22.00,27.30,1009.00,1016.10,1023.60,1.31,0.00,25.80,115.30,617.80,79.10,84.80,89.30,961.00,967.80,975.00,18.60,1.70,2.20,2.80,16.20,22.0

In [88]:
def add_30_mins(start_timestamp, end_timestamp):
    new_timestamp_list = []
    new_timestamp_list.append(start_timestamp.strftime("%Y-%m-%d %H:%M:%S"))
    new_timestamp = start_timestamp
    while new_timestamp < end_timestamp:
        new_timestamp = new_timestamp + timedelta(minutes = 60)
        if new_timestamp > end_timestamp:
            break
        else :
            new_timestamp_list.append(new_timestamp.strftime("%Y-%m-%d %H:%M:%S"))
    
    return new_timestamp_list

def create_time_groups(df):
    time_group_list = []
    df['round_time_of_first_Call'] = df['TIME_OF_FIRST_CALL'].dt.round('15min')
    for i in range(len(df)):
        time_intervals = add_30_mins(df['round_time_of_first_Call'][i], df['TIME_TO_RESTORE'][i])
        time_group_list.append(time_intervals)
    
    return time_group_list

time_intervals_group = create_time_groups(df_ads)

In [89]:
list_len = []
for i in range(len(time_intervals_group)):
    list_new = len(time_intervals_group[i])
    list_len.append(list_new)

In [90]:
len(list_len)

49989

In [91]:
final_timestamp = list(itertools.chain(*time_intervals_group))

In [92]:
print(len(list_len))
print(len(final_timestamp))

49989
468684


In [93]:
test = df_ads.copy(deep=True)
test['Times'] = list_len 

In [94]:
test_v1 = test.loc[test.index.repeat(test.Times)].reset_index(drop=True)
test_v1['Time_Bucket'] = final_timestamp

In [95]:
test_v1['Time_Bucket'] = pd.to_datetime(test_v1['Time_Bucket'], errors='coerce')

test_v1['Time_Since_Outage'] = (test_v1.Time_Bucket - test_v1.TIME_OF_FIRST_CALL).dt.total_seconds().div(60).round(4)

test_v1['TR'] = (test_v1.TIME_TO_RESTORE - test_v1.Time_Bucket).dt.total_seconds().div(60).round(4)

In [96]:
print(list(test_v1.columns))

['OUTAGE_ID', 'CIRCUIT', 'DEVICE', 'TIME_OF_FIRST_CALL', 'PHASE_A', 'PHASE_B', 'PHASE_C', 'PHASE_T', 'LFS_CUSTS', 'MED_CUSTS', 'EMR_CUSTS', 'UTL_CUSTS', 'MAJ_CUSTS', 'GOV_CUSTS', 'LRT_CUSTS', 'WIREDOWN_PTH', 'WIREDOWN_PTP', 'TREEONWIRE_PTH', 'TREEONWIRE_PTP', 'BROKENPOLE', 'FIREEXPLOSION', 'DAMAGEDTRANS', 'PRIORITY', 'CONFIDENCE', 'CUSTSTILLOUT', 'TTR', 'MAJOR', 'TORD_False', 'TORD_True', 'ACCOUNT_TYPE_COM', 'ACCOUNT_TYPE_CUS', 'ACCOUNT_TYPE_GOV', 'ACCOUNT_TYPE_IND', 'ACCOUNT_TYPE_RES', 'STORM_TYPE', 'EVENT', 'Day_flag', 'Hour_Sin', 'Hour_Cos', 'DATE', 'RANK_SUBSEQUENT_OTG', 'weekday', 'Weekend_flag', 'Marker_Location', 'Outages_in_last_1hr', 'Outages_in_last_2hr', 'Outages_in_last_3hr', 'Outages_in_last_4hr', 'Outages_in_last_5hr', 'Outages_in_last_6hr', 'Outages_in_last_7hr', 'Outages_in_last_8hr', 'Outages_in_last_9hr', 'Outages_in_last_10hr', 'Min_Distance_service_Center', 'Nearest_service_center', 'cldCvrMin', 'cldCvrAvg', 'cldCvrMax', 'dewPtMin', 'dewPtAvg', 'dewPtMax', 'feelsLik

In [97]:
test_v1.sort_values(by=['TIME_OF_FIRST_CALL'], inplace=True)
test_v1.reset_index(drop=True, inplace=True)

In [98]:
test_v1[['OUTAGE_ID', 'TIME_OF_FIRST_CALL', 'TIME_TO_RESTORE', 'TR', 'Times', 'Time_Bucket', 'Outages_in_last_1hr', 'Outages_in_last_2hr', 'Time_Since_Outage']].head(20)

,OUTAGE_ID,TIME_OF_FIRST_CALL,TIME_TO_RESTORE,TR,Times,Time_Bucket,Outages_in_last_1hr,Outages_in_last_2hr,Time_Since_Outage
0,598691,2007-02-13 01:34:17,2007-02-13 07:25:41.000,355.68,6,2007-02-13 01:30:00,0,0,-4.28
1,598691,2007-02-13 01:34:17,2007-02-13 07:25:41.000,295.68,6,2007-02-13 02:30:00,0,0,55.72
2,598691,2007-02-13 01:34:17,2007-02-13 07:25:41.000,235.68,6,2007-02-13 03:30:00,0,0,115.72
3,598691,2007-02-13 01:34:17,2007-02-13 07:25:41.000,175.68,6,2007-02-13 04:30:00,0,0,175.72
4,598691,2007-02-13 01:34:17,2007-02-13 07:25:41.000,115.68,6,2007-02-13 05:30:00,0,0,235.72
5,598691,2007-02-13 01:34:17,2007-02-13 07:25:41.000,55.68,6,2007-02-13 06:30:00,0,0,295.72
6,598695,2007-02-13 07:02:48,2007-02-13 10:11:37.200,11.62,4,2007-02-13 10:00:00,0,0,177.20
7,598695,2007-02-13 07:02:48,2007-02-13 10:11:37.200,71.62,4,2007-02-13 09:00:00,0,0,117.20
8,598695,2007-02-13 07:02:48,2007-02-13 10:11:37.200,191.62,4,2007-02-13 07:00:00,0,0,-2.80
9,598695,2007-02-13 07:02:48,2007-02-13 10:11:37.200,131.62,4,2007-02-13 08:00:00,0,0,57.20


In [99]:
test_v1 = test_v1[test_v1.Time_Since_Outage >= 0]
test_v1['Times']  = test_v1['Times'] - 1
test_v1[['OUTAGE_ID', 'TIME_OF_FIRST_CALL', 'TIME_TO_RESTORE', 'TR', 'Times', 'Time_Bucket', 'Outages_in_last_1hr', 'Outages_in_last_2hr', 'Time_Since_Outage']].head(20)

,OUTAGE_ID,TIME_OF_FIRST_CALL,TIME_TO_RESTORE,TR,Times,Time_Bucket,Outages_in_last_1hr,Outages_in_last_2hr,Time_Since_Outage
1,598691,2007-02-13 01:34:17,2007-02-13 07:25:41.000,295.68,5,2007-02-13 02:30:00,0,0,55.72
2,598691,2007-02-13 01:34:17,2007-02-13 07:25:41.000,235.68,5,2007-02-13 03:30:00,0,0,115.72
3,598691,2007-02-13 01:34:17,2007-02-13 07:25:41.000,175.68,5,2007-02-13 04:30:00,0,0,175.72
4,598691,2007-02-13 01:34:17,2007-02-13 07:25:41.000,115.68,5,2007-02-13 05:30:00,0,0,235.72
5,598691,2007-02-13 01:34:17,2007-02-13 07:25:41.000,55.68,5,2007-02-13 06:30:00,0,0,295.72
6,598695,2007-02-13 07:02:48,2007-02-13 10:11:37.200,11.62,3,2007-02-13 10:00:00,0,0,177.20
7,598695,2007-02-13 07:02:48,2007-02-13 10:11:37.200,71.62,3,2007-02-13 09:00:00,0,0,117.20
9,598695,2007-02-13 07:02:48,2007-02-13 10:11:37.200,131.62,3,2007-02-13 08:00:00,0,0,57.20
11,598696,2007-02-13 07:33:14,2007-02-13 09:00:42.200,30.70,1,2007-02-13 08:30:00,1,1,56.77
13,598701,2007-02-13 09:06:06,2007-02-13 10:32:04.200,32.07,1,2007-02-13 10:00:00,0,1,53.90


In [100]:
print(list(test_v1.columns))

['OUTAGE_ID', 'CIRCUIT', 'DEVICE', 'TIME_OF_FIRST_CALL', 'PHASE_A', 'PHASE_B', 'PHASE_C', 'PHASE_T', 'LFS_CUSTS', 'MED_CUSTS', 'EMR_CUSTS', 'UTL_CUSTS', 'MAJ_CUSTS', 'GOV_CUSTS', 'LRT_CUSTS', 'WIREDOWN_PTH', 'WIREDOWN_PTP', 'TREEONWIRE_PTH', 'TREEONWIRE_PTP', 'BROKENPOLE', 'FIREEXPLOSION', 'DAMAGEDTRANS', 'PRIORITY', 'CONFIDENCE', 'CUSTSTILLOUT', 'TTR', 'MAJOR', 'TORD_False', 'TORD_True', 'ACCOUNT_TYPE_COM', 'ACCOUNT_TYPE_CUS', 'ACCOUNT_TYPE_GOV', 'ACCOUNT_TYPE_IND', 'ACCOUNT_TYPE_RES', 'STORM_TYPE', 'EVENT', 'Day_flag', 'Hour_Sin', 'Hour_Cos', 'DATE', 'RANK_SUBSEQUENT_OTG', 'weekday', 'Weekend_flag', 'Marker_Location', 'Outages_in_last_1hr', 'Outages_in_last_2hr', 'Outages_in_last_3hr', 'Outages_in_last_4hr', 'Outages_in_last_5hr', 'Outages_in_last_6hr', 'Outages_in_last_7hr', 'Outages_in_last_8hr', 'Outages_in_last_9hr', 'Outages_in_last_10hr', 'Min_Distance_service_Center', 'Nearest_service_center', 'cldCvrMin', 'cldCvrAvg', 'cldCvrMax', 'dewPtMin', 'dewPtAvg', 'dewPtMax', 'feelsLik

In [101]:
test_v1['REP_OUTAGE_ID'] = test_v1['OUTAGE_ID'].astype(str) + test_v1['Time_Bucket'].astype(str).str[8:16]
outage_Count_df = test_v1[['OUTAGE_ID','TIME_OF_FIRST_CALL']].drop_duplicates()

In [102]:
test_v1.shape

(443529, 153)

In [106]:
def filter_low(group):
    group.reset_index(inplace = True)
    if(group.Time_Since_Outage.max() <= 60):
        return group.OUTAGE_ID[0]
    else:
        return 1
outage_id_rem = test_v1.groupby('OUTAGE_ID').apply(filter_low).reset_index(drop = True)

In [107]:
test_v1 = test_v1[~test_v1.OUTAGE_ID.isin(outage_id_rem)]

In [112]:
x = test_v1[test_v1.TIME_OF_FIRST_CALL.dt.year > 2011]
x.STORM_TYPE.count()

229092

In [115]:
test_v1[test_v1.TIME_OF_FIRST_CALL.dt.year > 2011].to_csv('gs://aes-analytics-0002-curated/Outage_Restoration/DPL_Repredictions/DPL_Master_dataset_reprediction_v1.csv')